In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import bz2
import json
from tqdm import tqdm

Mounted at /content/drive


In [2]:
%cd "/content/drive/My Drive"
!ls

/content/drive/My Drive
 ada_project	    Doomed-to-learn   Quotebank
'Colab Notebooks'   lego_project      Routine.xlsx


In [3]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 4.2 MB/s 
     |████████████████████████████████| 3.3 MB 54.0 MB/s 
     |████████████████████████████████| 895 kB 80.4 MB/s 
     |████████████████████████████████| 596 kB 88.8 MB/s 
     |████████████████████████████████| 61 kB 594 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
def create_dataframe_from_json_bz2(path_file):
    with bz2.open(path_file, 'rb') as file:
        df = pd.read_json(file, lines=True)
    return df

In [5]:
years = ['2018', '2019', '2020']
data = [create_dataframe_from_json_bz2('ada_project/quotes-{}-china_trade.json.bz2'.format(year)) for year in years]
df = pd.concat(data, ignore_index=True)
df.head()

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2018-04-13-001441,A Digital Free Trade Zone between China and Ma...,Peter Wong,"[Q7177768, Q7177774, Q7177776, Q9456225]",2018-04-13 19:30:55,1,"[[Peter Wong, 0.782], [None, 0.218]]",[http://nst.com.my/business/2018/04/356893/chi...,E
1,2018-03-23-003097,A rough week for the markets... as fears of a ...,None,[],2018-03-23 10:28:51,2,"[[None, 0.9112], [President Donald Trump, 0.08...",[http://www.breitbart.com/news/world-stock-mar...,E
2,2018-04-08-011525,"Every day of the week China, comes into our ho...",Peter Navarro,[Q7176052],2018-04-08 04:00:00,17,"[[Peter Navarro, 0.6696], [None, 0.208], [LARR...",[http://dailylocal.com/general-news/20180408/a...,E
3,2018-05-14-023366,For the President to become suddenly concerned...,Jonathan Fenby,[Q15072639],2018-05-14 20:43:32,2,"[[Jonathan Fenby, 0.8834], [None, 0.1166]]",[https://www.fxstreet.com/news/wall-street-dow...,E
4,2018-11-29-030995,"he would be able to strike a chord with Xi, wh...",A. Khan,[Q54946635],2018-11-29 07:11:20,1,"[[A. Khan, 0.6289], [Narendra Modi, 0.204], [N...",[http://asia.nikkei.com/Politics/International...,E


In [6]:
df.drop(['qids', 'probas', 'urls', 'phase'], axis=1, inplace=True)
df.date = df.date.dt.strftime('%d/%m/%Y')
df.head()

,quoteID,quotation,speaker,date,numOccurrences
0,2018-04-13-001441,A Digital Free Trade Zone between China and Ma...,Peter Wong,13/04/2018,1
1,2018-03-23-003097,A rough week for the markets... as fears of a ...,None,23/03/2018,2
2,2018-04-08-011525,"Every day of the week China, comes into our ho...",Peter Navarro,08/04/2018,17
3,2018-05-14-023366,For the President to become suddenly concerned...,Jonathan Fenby,14/05/2018,2
4,2018-11-29-030995,"he would be able to strike a chord with Xi, wh...",A. Khan,29/11/2018,1


In [7]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification")

hypothesis_template = 'The sentiment of this quote is {}.'
# labels = ['positive', 'negative']
labels = ['calm', 'alert', 'sure', 'vital', 'kind', 'happy']

sentiment = []

sequences = df['quotation'].to_numpy()
for idx, quotation in tqdm(sequences):
    pred = classifier(quotation, labels, hypothesis_template=hypothesis_template, multi_label=True)  # Set multi_label=True
    sentiment.append(pred)

# 36 591 quotes to process

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)


Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.


RuntimeError: ignored

In [ ]:
# with open('ada_project/china_trade_zero_shot_sent_pos_neg.json', 'w') as file:
#     json.dump(sentiment, file)

In [ ]:
with open('ada_project/china_trade_zero_shot_sent_extra_features.json', 'w') as file:  # Set yout own path
    json.dump(sentiment, file)

In [ ]:
# df.to_json('ada_project/china_trade_zero_shot_sentiment.json', orient='records', lines=True)